In [13]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [4]:
#get data from grammar's csv file of a single type format
cluster_info = np.loadtxt(open("test_files/test.csv", "r"), delimiter=",", skiprows=1)
cluster_info

array([[25.  ,  2.  ,  4.25,  3.  ],
       [21.  ,  3.  ,  1.  ,  2.  ],
       [17.  ,  4.  ,  3.25, -3.  ],
       [22.  ,  5.  ,  6.75, -4.  ]])

In [5]:
k_size = int(len(cluster_info)/2) #k_size is arbitrarily set, equal to the number of clusters
print(k_size)
kmeans = KMeans(n_clusters=k_size).fit(cluster_info)

2


In [6]:
#outputs which cluster number the data point is placed into
kmeans.labels_

array([1, 1, 0, 0], dtype=int32)

In [7]:
#kmeans.predict(fill in with our own 7 unit vector data), will classify it into cluster

In [8]:
class Node:
    def __init__(self, node_num):
        self.node_num = node_num
        self.s_exp = [] #list of s-exp-ids
        self.cpt = {} 
        self.count = 0
    
    def add_exp(self, s):
        self.s_exp.append(s)

In [36]:
#dictionary that maps s-exp-id to: s-exp, song-id (which song), song-index (position in song)
s_exp_dict = {} 

#assume s_exp_labels is grammar output with 3 columns: s-exp, song-id, song-index
#s_exp_labels = np.loadtxt(open("test_files/test_exp.csv", "r"), dtype="str", delimiter=",", skiprows=1)
s_exp_labels = pd.read_csv('test_files/test_exp.csv')
s_exp_labels

,s-exp,song-id,song-index
0,asdf,1,1
1,gdnk,1,2
2,hjkl,1,3
3,sifn,1,4
4,husk,2,1
5,slei,2,2
6,wert,3,1


In [37]:
# for expression in s_exp_labels:
#     s_exp_dict[expression[0]] = [expression[1], expression[2], expression[3]]
for index, row in s_exp_labels.iterrows():
    s_exp_dict[index] = [row['s-exp'], row['song-id'], row['song-index']]

s_exp_dict

{0: ['asdf', 1, 1],
 1: ['gdnk', 1, 2],
 2: ['hjkl', 1, 3],
 3: ['sifn', 1, 4],
 4: ['husk', 2, 1],
 5: ['slei', 2, 2],
 6: ['wert', 3, 1]}

In [68]:
node_objects = [] #list of nodes for the Markov chain

for cluster_num in range(k_size):
    node_objects.append(Node(cluster_num)) #initialize 1 node for each cluster number
    
for i in range(0,len(kmeans.labels_)): #iterating through all the data points
    cluster_num = kmeans.labels_[i] #which cluster num the data point corresponds to
    #adding s-exp, song-id (which song), song-index (position in song) to the node object
    node_objects[cluster_num].add_exp(s_exp_dict[i]) #why string??? (previously)

In [69]:
#create cpt
for outer_node in node_objects:
    for inner_node in node_objects:
        outer_node.cpt[inner_node.node_num] = 0
        if (outer_node != inner_node):
            for s_expression_outer in outer_node.s_exp:
                for s_expression_inner in inner_node.s_exp:
                    if s_expression_outer[1] == s_expression_inner[1] and s_expression_inner[2] - s_expression_outer[2] == 1:
                        outer_node.count += 1
                        outer_node.cpt[inner_node.node_num] += 1
    outer_node.cpt = {k:v/outer_node.count for k, v in outer_node.cpt.items() if outer_node.count != 0}
print(outer_node.cpt)

{0: 1.0, 1: 0.0}
